#StructType

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Struct_Type") \
    .getOrCreate()

from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType, LongType

)

In [ ]:
raw_data = [
    ("U001","Abhishek",28,"Hyderabad",50000),
    ("U002","Neha",32,"Delhi",62000),
    ("U003","Ravi",25,"Bangalore",45000),
    ("U004","Pooja",29,"Mumbai",58000)
]

In [ ]:
user_schema = StructType ([
    StructField("user_id", StringType(), nullable=False),
    StructField("name", StringType(), nullable=True),
    StructField("age", IntegerType(), nullable=True),
    StructField("city", StringType(), nullable=True),
    StructField("salary", LongType(), nullable=True),
])

In [ ]:
df_users= spark.createDataFrame(raw_data, schema=user_schema)
df_users.printSchema()
df_users.show()


root
 |-- user_id: string (nullable = false)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- salary: long (nullable = true)

+-------+--------+---+---------+------+
|user_id|    name|age|     city|salary|
+-------+--------+---+---------+------+
|   U001|Abhishek| 28|Hyderabad| 50000|
|   U002|    Neha| 32|    Delhi| 62000|
|   U003|    Ravi| 25|Bangalore| 45000|
|   U004|   Pooja| 29|   Mumbai| 58000|
+-------+--------+---+---------+------+



In [ ]:
#Wrong Data
raw_data2=[
    ("U005","Amit","Thirty","Chennai",40000)
]

df_users = spark.createDataFrame(raw_data2, schema=user_schema)

#For Arrays - ArrayType

In [ ]:
from pyspark.sql.types import ArrayType

In [ ]:
interest_data = [
    ("U001",["AI","ML","Cloud"]),
    ("U002",["Testing","Automation"]),
    ("U003",["Data Engineering","Spark","Kafka"]),
    ("U004",["UI/UX"])
]

In [ ]:
interest_schema = StructType([
    StructField("user_id", StringType(),False),
    StructField("interests", ArrayType(StringType()),True)
])

In [ ]:
df_interests= spark.createDataFrame(interest_data, schema=interest_schema)
df_interests.printSchema()
df_interests.show()

root
 |-- user_id: string (nullable = false)
 |-- interests: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------+--------------------+
|user_id|           interests|
+-------+--------------------+
|   U001|     [AI, ML, Cloud]|
|   U002|[Testing, Automat...|
|   U003|[Data Engineering...|
|   U004|             [UI/UX]|
+-------+--------------------+



Explode

In [ ]:
from pyspark.sql.functions import explode

df_interests.select (
    "user_id",
    explode("interests").alias("interest")
).show()

+-------+----------------+
|user_id|        interest|
+-------+----------------+
|   U001|              AI|
|   U001|              ML|
|   U001|           Cloud|
|   U002|         Testing|
|   U002|      Automation|
|   U003|Data Engineering|
|   U003|           Spark|
|   U003|           Kafka|
|   U004|           UI/UX|
+-------+----------------+



#MapType

In [ ]:
from pyspark.sql.types import MapType

In [ ]:
device_data =[
    ("U001",{"mobile":120,"laptop":300}),
    ("U002",{"tablet":80}),
    ("U003",{"mobile":200,"desktop":400}),
    ("U004",{"laptop":250})
]

In [ ]:
device_schema = StructType ([
    StructField("user_id", StringType(), False),
    StructField("device_usage", MapType(StringType(), IntegerType()),True)
])

In [ ]:
df_devices = spark.createDataFrame(device_data,device_schema)
df_devices.printSchema()
df_devices.show(truncate=False)

root
 |-- user_id: string (nullable = false)
 |-- device_usage: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)

+-------+-------------------------------+
|user_id|device_usage                   |
+-------+-------------------------------+
|U001   |{mobile -> 120, laptop -> 300} |
|U002   |{tablet -> 80}                 |
|U003   |{mobile -> 200, desktop -> 400}|
|U004   |{laptop -> 250}                |
+-------+-------------------------------+



#Nested Schema

In [ ]:
nested_data = [
    ("U001",("Hyderabad","Telangana",500081)),
    ("U002",("Delhi","Delhi",110001)),
    ("U003",("Bangalore","Karnataka",560001))
]

In [ ]:
address_schema = StructType([
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("pincode", IntegerType(), True)

])

profile_schema = StructType([
    StructField("user_id",StringType(),False),
    StructField("address", address_schema, True)
])

In [ ]:
df_profiles = spark.createDataFrame(nested_data, profile_schema)
df_profiles.printSchema()
df_profiles.show(truncate=False)

root
 |-- user_id: string (nullable = false)
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- pincode: integer (nullable = true)

+-------+------------------------------+
|user_id|address                       |
+-------+------------------------------+
|U001   |{Hyderabad, Telangana, 500081}|
|U002   |{Delhi, Delhi, 110001}        |
|U003   |{Bangalore, Karnataka, 560001}|
+-------+------------------------------+



To select in this nested schema

In [ ]:
df_profiles.select(
    "user_id",
    "address.city",
    "address.state"
).show()

+-------+---------+---------+
|user_id|     city|    state|
+-------+---------+---------+
|   U001|Hyderabad|Telangana|
|   U002|    Delhi|    Delhi|
|   U003|Bangalore|Karnataka|
+-------+---------+---------+



To create a new column with new data type

In [ ]:
from pyspark.sql.functions import col

df_users.withColumn(
    "salary_int",
    col("salary").cast("int")
)

DataFrame[user_id: string, name: string, age: int, city: string, salary: bigint, salary_int: int]

To convert to date

In [ ]:
from pyspark.sql.functions import to_date

df_orders.withColumn(
    "order_date",
    to_date("order_date", "yyyy-mm-dd")
)